<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5> Aarhus Criminal Correlation </font></h1>

## Introduction

Aarhus is a small city in the small country Denmark, the smallest scandinavian country. 
The city is the second largest city in Denmark, second to Copenhagen, and is undergoing a similar internasionalisation as the larger capital. 

Denmark as a whole is a country with very few criminal cases, and the hypothesis that is being tested today is, how is criminal behavior distributed along the city and is what effects does the economic classes have on the criminal record. 

## Table of Contents

1. <a href="#item1">The Data</a>
2. <a href="#item2">Explore a Given Venue</a>  
3. <a href="#item3">Explore a User</a>  
4. <a href="#item4">Foursquare API Explore Function</a>  
5. <a href="#item5">Get Trending Venues</a>  

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# !conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0



geopy-2.0.0          | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: d

### Define Foursquare Credentials and Version

In [2]:
CLIENT_ID = 'CZ0ZNBE1HBE1LJOKEFMMKWKRZE05ZGCU5SIRDVOLD2SSQKG0' # your Foursquare ID
CLIENT_SECRET = 'PD2PXGQJGPP0QULJ5HBO1F3JMJUNUTBBHD04LV1SCGGAP1GB' # your Foursquare Secret
VERSION = '20200704'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CZ0ZNBE1HBE1LJOKEFMMKWKRZE05ZGCU5SIRDVOLD2SSQKG0
CLIENT_SECRET:PD2PXGQJGPP0QULJ5HBO1F3JMJUNUTBBHD04LV1SCGGAP1GB


### 1. The Data

The data is build on population over several districts. 
these districts will then be showed in a map where econimic distribution will be shared. 

but first, let's see Aarhus as the City that it is.

First the city population data will be observed

In [3]:
df_aarhus = pd.read_excel("hele-befolkningen-1-januar-2013.xls", 'Tabel 6')

df_aarhus.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Tabel 6. Befolkningen i Århus Kommune den 1. j...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Lokalsamfund,Kvinder,Mænd,I alt,Heraf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,0-2 år,3-5 år,6-15 år,16-19 år,20-24 år,25-64 år,65 år -,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,City,5092,5711,10803,251,124,287,256,3086,6126,673,NaN,10803.0
8,Frederiksbjerg,8760,7719,16479,611,328,707,379,3266,9714,1474,NaN,16479.0
9,Vesterbro,10516,9919,20435,665,327,639,428,5624,11202,1550,NaN,20435.0


The data needs some cleaning as it is clear that the import had some issues with defining the Columns, cleaning the dataset is simple. 
first the rows 0-6 needs to be deleted. but the values from row 3, indicates the columnsnames of the first three columns, therefore. we do the following:



In [4]:
df_aarhus = df_aarhus.drop([0,1,2])
df_aarhus.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
3,Lokalsamfund,Kvinder,Mænd,I alt,Heraf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,0-2 år,3-5 år,6-15 år,16-19 år,20-24 år,25-64 år,65 år -,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,City,5092,5711,10803,251,124,287,256,3086,6126,673,NaN,10803.0


In [5]:
columns1 = df_aarhus.iloc[0,0]
columns2 = df_aarhus.iloc[0,1]
columns3 = df_aarhus.iloc[0,2]
columns4 = df_aarhus.iloc[0,3]
# Age columns
columns5 = df_aarhus.iloc[2,4]
columns6 = df_aarhus.iloc[2,5]
columns7 = df_aarhus.iloc[2,6]
columns8 = df_aarhus.iloc[2,7]
columns9 = df_aarhus.iloc[2,8]
columns10 = df_aarhus.iloc[2,9]
columns11 = df_aarhus.iloc[2,10]
columns12 = df_aarhus.iloc[2,11]
columns13 = df_aarhus.iloc[2,12]
df_aarhus.columns = [columns1,columns2,columns3,columns4,columns5,columns6,columns7,columns8,columns9,columns10,columns11,columns12,columns13]

In [6]:
df_aarhus.head()
df_aarhus = df_aarhus = df_aarhus.drop([3,4,5,6])

In [7]:
df_aarhus = df_aarhus = df_aarhus.drop([36,37,38,39,40,41])
df_aarhus.dropna(axis=1, how='all', inplace=True)
# More cleaning of the data

In [8]:
# Renaming the aarhus Ø data.
df_aarhus.iloc[4,0] = 'Aarhus Ø'
df_aarhus.iloc[5,0] = 'Holme'
df_aarhus.iloc[14,0] = 'Brabrand'
df_aarhus.iloc[15,0] = 'Tilst'
df_aarhus.iloc[16,0] = 'Skejby'
df_aarhus.iloc[17,0] = 'lystrup'
df_aarhus.iloc[23,0] = 'Sabro'
df_aarhus.iloc[24,0] = 'Trige'
df_aarhus.iloc[25,0] = 'Mejlby'
df_aarhus.iloc[27,0] = 'Skødstrup'






In [9]:
df_aarhus.iloc[:,11] = df_aarhus.iloc[:,3]
df_aarhus.columns = df_aarhus.columns.fillna('Total')
df_aarhus

,Lokalsamfund,Kvinder,Mænd,I alt,0-2 år,3-5 år,6-15 år,16-19 år,20-24 år,25-64 år,65 år -,Total
7,City,5092,5711,10803,251,124,287,256,3086,6126,673,10803
8,Frederiksbjerg,8760,7719,16479,611,328,707,379,3266,9714,1474,16479
9,Vesterbro,10516,9919,20435,665,327,639,428,5624,11202,1550,20435
10,Trøjborg,5863,5272,11135,357,159,300,241,3041,6376,661,11135
11,Aarhus Ø,352,312,664,6,4,11,36,357,236,14,664
12,Holme,12353,11368,23721,791,752,2773,1252,1689,12128,4336,23721
13,Viby,15388,15033,30421,985,831,2888,1386,4130,15408,4793,30421
14,Hasle,11564,11417,22981,940,835,2331,1089,3607,11451,2728,22981
15,Christiansbjerg,9418,9014,18432,576,490,1501,738,3536,9541,2050,18432
16,Vejlby-Risskov,12828,11832,24660,798,830,2963,1411,2370,12291,3997,24660


In [10]:
import folium # map rendering library

latitude = 56.162939
longitude = 10.203921

# create map of Toronto using latitude and longitude values
map_aarhus = folium.Map(location=[latitude, longitude], zoom_start=11)

    
map_aarhus

In [11]:
len(df_aarhus)

29

In [12]:
# Get lattitude and longitude data of aarhus 
from geopy.geocoders import Nominatim

Latitude_aar = []
Longitude_aar = []

for i in range(len(df_aarhus)): 

    if (i == 28) :
        cityName = 'Aarhus'
    else :
        cityName = df_aarhus.iloc[i,0]
        
    address=cityName + ',Aarhus'
    geolocator = Nominatim(user_agent="Abdalla")
    location = geolocator.geocode(address)
    Latitude_aar.append(location.latitude)
    Longitude_aar.append(location.longitude)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28


In [13]:
df_aarhus['Latitude'] = Latitude_aar
df_aarhus['Longitude'] = Longitude_aar

In [14]:
# add markers to map
for lat, lng, community, total  in zip(df_aarhus['Latitude'], df_aarhus['Longitude'], df_aarhus['Lokalsamfund'], df_aarhus['Total']):
    label = '{}, {}'.format(community, total)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_aarhus)  

In [15]:
map_aarhus

Importing K-Means from Sktilearn in order to cluister the communities together: 

In [21]:
# Importing KMeans from ML package sklearn and, cm & colors from matplotlib 
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors





# grouping Toronto Data

aarhus_grouped = df_aarhus.groupby('Lokalsamfund').mean().reset_index()
Cluster_aarhus = aarhus_grouped.drop('Lokalsamfund', 1)
Cluster_aarhus

# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Cluster_aarhus)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

aarhus_grouped.insert(0, 'Cluster Labels', kmeans.labels_)



In [22]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(aarhus_grouped['Latitude'], aarhus_grouped['Longitude'], aarhus_grouped['Lokalsamfund'], aarhus_grouped['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Zooming into the purple cluster. let's examine what venues are close by. 

In [68]:
search_query = 'Vesterbro,aarhus'
latitude_FS = df_aarhus['Latitude'][9]
longitude_FS =  df_aarhus['Longitude'][9]
radius = 500


url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_FS, longitude_FS, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=CZ0ZNBE1HBE1LJOKEFMMKWKRZE05ZGCU5SIRDVOLD2SSQKG0&client_secret=PD2PXGQJGPP0QULJ5HBO1F3JMJUNUTBBHD04LV1SCGGAP1GB&ll=56.15820975,10.198986218151024&v=20200704&query=Vesterbro,aarhus&radius=500&limit=30'

In [82]:
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
venue_df = pd.json_normalize(venues)

venue_df.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood,location.crossStreet
0,4c17f46a838020a1ca76e361,Vesterbro Torv,"[{'id': '4bf58dd8d48988d164941735', 'name': 'P...",v-1595851915,False,Vesterbro Torv,56.158183,10.198681,"[{'label': 'display', 'lat': 56.15818323105313...",19,8000,DK,Århus,Region Midtjylland,Danmark,"[Vesterbro Torv, 8000 Århus, Danmark]",NaN,NaN,NaN
1,4c03cbe30d0e0f477d00039a,Bus Stop Vesterbro Torv,"[{'id': '4bf58dd8d48988d1fe931735', 'name': 'B...",v-1595851915,False,NaN,56.158145,10.198767,"[{'label': 'display', 'lat': 56.15814547696767...",15,NaN,DK,NaN,NaN,Danmark,[Danmark],NaN,NaN,NaN
2,4dde1b50183877913995a3dd,Vesterbro Kiosk,"[{'id': '4d4b7105d754a06378d81259', 'name': 'S...",v-1595851915,False,NaN,56.158101,10.198957,"[{'label': 'display', 'lat': 56.15810115000318...",12,NaN,DK,Århus,Region Midtjylland,Danmark,"[Århus, Danmark]",NaN,NaN,NaN
3,511fa087e4b0075006ef1b41,Aarhus,"[{'id': '50aa9e094b90af0d42d5de0d', 'name': 'C...",v-1595851915,False,NaN,56.158171,10.198832,NaN,10,8000,DK,NaN,Region Midtjylland,Danmark,"[8000, Danmark]",344343454,NaN,NaN
4,59a44a2c8173cb1a33a95f92,Vesterbro Falls,"[{'id': '56aa371be4b08b9a8d573560', 'name': 'W...",v-1595851915,False,NaN,56.158239,10.199401,"[{'label': 'display', 'lat': 56.158239, 'lng':...",25,8000,DK,Århus,Region Midtjylland,Danmark,"[8000 Århus, Danmark]",NaN,NaN,NaN


We need to change categories and filter the data so we only have relervant data associated with location.

In [85]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in venue_df.columns if col.startswith('location.')] + ['id']
dataframe_filtered = venue_df.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()



,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,crossStreet,id
0,Vesterbro Torv,Plaza,Vesterbro Torv,56.158183,10.198681,"[{'label': 'display', 'lat': 56.15818323105313...",19,8000,DK,Århus,Region Midtjylland,Danmark,"[Vesterbro Torv, 8000 Århus, Danmark]",NaN,NaN,4c17f46a838020a1ca76e361
1,Bus Stop Vesterbro Torv,Bus Station,NaN,56.158145,10.198767,"[{'label': 'display', 'lat': 56.15814547696767...",15,NaN,DK,NaN,NaN,Danmark,[Danmark],NaN,NaN,4c03cbe30d0e0f477d00039a
2,Vesterbro Kiosk,Shop & Service,NaN,56.158101,10.198957,"[{'label': 'display', 'lat': 56.15810115000318...",12,NaN,DK,Århus,Region Midtjylland,Danmark,"[Århus, Danmark]",NaN,NaN,4dde1b50183877913995a3dd
3,Aarhus,City,NaN,56.158171,10.198832,NaN,10,8000,DK,NaN,Region Midtjylland,Danmark,"[8000, Danmark]",NaN,NaN,511fa087e4b0075006ef1b41
4,Vesterbro Falls,Waterfall,NaN,56.158239,10.199401,"[{'label': 'display', 'lat': 56.158239, 'lng':...",25,8000,DK,Århus,Region Midtjylland,Danmark,"[8000 Århus, Danmark]",NaN,NaN,59a44a2c8173cb1a33a95f92


In [89]:
# Removing some of the unnecesary columns. 
del dataframe_filtered['labeledLatLngs']
del dataframe_filtered['crossStreet']
del dataframe_filtered['neighborhood']


In [96]:
dataframe_filtered.head()

,name,categories,address,lat,lng,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Vesterbro Torv,Plaza,Vesterbro Torv,56.158183,10.198681,19,8000,DK,Århus,Region Midtjylland,Danmark,"[Vesterbro Torv, 8000 Århus, Danmark]",4c17f46a838020a1ca76e361
1,Bus Stop Vesterbro Torv,Bus Station,NaN,56.158145,10.198767,15,NaN,DK,NaN,NaN,Danmark,[Danmark],4c03cbe30d0e0f477d00039a
2,Vesterbro Kiosk,Shop & Service,NaN,56.158101,10.198957,12,NaN,DK,Århus,Region Midtjylland,Danmark,"[Århus, Danmark]",4dde1b50183877913995a3dd
3,Aarhus,City,NaN,56.158171,10.198832,10,8000,DK,NaN,Region Midtjylland,Danmark,"[8000, Danmark]",511fa087e4b0075006ef1b41
4,Vesterbro Falls,Waterfall,NaN,56.158239,10.199401,25,8000,DK,Århus,Region Midtjylland,Danmark,"[8000 Århus, Danmark]",59a44a2c8173cb1a33a95f92


In [98]:
map_vesterbro = folium.Map(location=[latitude_FS, longitude_FS], zoom_start=20)



for lat, lng, categories, name  in zip(dataframe_filtered['lat'], dataframe_filtered['lng'], dataframe_filtered['categories'], dataframe_filtered['name']):
    label = '{}, {}'.format(categories, total)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_vesterbro)  
    
    map_vesterbro

In [99]:
    map_vesterbro